# Lab 5: Hitters (Python)

In the computational section of this Lab you will consider the baseball dataset found in the file hitters.csv. This dataset records the salary of 𝑛 = 263 Major League Baseball players during the 1987 season as well as 𝑞 = 19 statistics associated with the performance of each player during the previous season. Specifically, the dataset contains observations from the following variables:
* AtBat: Number of times at bat in 1986
* Hits: Number of hits in 1986
* HmRun: Number of home runs in 1986
* Runs: Number of runs in 1986
* RBI: Number of runs batted in in 1986
* Walks: Number of walks in 1986
* Years: Number of years in the major leagues
* CAtBat: Number of times at bat during his career
* CHits: Number of hits during his career
* CHmRun: Number of home runs during his career
* CRuns: Number of runs during his career
* CRBI: Number of runs batted in during his career
* CWalks: Number of walks during his career
* League: A categorical variable with levels A (for American) and N (for National) indicating the player’s league at the end of 1986
* Division: A factor with levels E (for East) and W (for West) indicating the player’s division at the end of 1986
* PutOuts: Number of put outs in 1986
* Assists: Number of assists in 1986
* Errors: Number of errors in 1986
* Salary: 1987 annual salary on opening day in thousands of dollars
* NewLeague: A factor with levels A and N indicating the player’s league at the beginning of 1987

In [1]:
import os
import pandas as pd
import statsmodels.formula.api as smf
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.cross_validation import KFold
data = pd.read_csv('hitters.csv')

/Users/jencruz/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


(a) Calculate the variance inflation factor (VIF) for each of the explanatory variables.Comment on whether multicollinearity appears to be an issue. If it is, identify the three explanatory variables that are most seriously affected by the issue.

In [2]:
League = pd.get_dummies(data['League'])
Division = pd.get_dummies(data['Division'])
NewLeague = pd.get_dummies(data['NewLeague'])
data = data.drop(['League', 'Division', 'NewLeague'], axis = 1)
data = pd.concat([data, League['N'], Division['W'], NewLeague['N']], axis = 1)
data.columns = ['AtBat', 'Hits', 'HmRun', 'Runs', 'RBI', 'Walks', 'Years', 'CAtBat', 'CHits', 'CHmRun', 'CRuns', 'CRBI', 'CWalks', 'PutOuts', 'Assists', 'Errors', 'Salary', 'League', 'Division', 'NewLeague']
X = data.drop(['Salary'], axis = 1)
y = data['Salary']
col = X.columns
vif = np.repeat(0,19)
for i in range(19):    
    m = smf.ols(col[i] + '~' + col[0] + '+' + col[1] + '+' + col[2] + '+' + col[3] + '+' + col[4] + '+' + col[5] + '+' + col[7] + '+' + col[7] + '+' + col[8] + '+' + col[9] + '+' + col[10] + '+' + col[11] + '+' + col[12] + '+' + col[13] + '+' + col[14] + '+' + col[15] + '+' + col[16] + '+' + col[17] + '+' + col[18] + '-' + col[i], data = X).fit()
    vif[i] = 1/(1-m.rsquared)
vif = pd.DataFrame(vif, col)
vif.sort_values([0])

,0
Division,1
PutOuts,1
Errors,2
Assists,2
NewLeague,4
League,4
Walks,4
HmRun,7
Years,9
RBI,11


Based on the VIFs that were calculated above, we can see that multicollinearity is an issue. The top variables that are seriously affectd by this issue are: CRBI, CRuns, CAtBat, and Chits. These all have large VIFs, but this is not suprising since the # of runs scored and RBIs ina player's career are both going to be highly correlated with the # of hits the player gets in their career. And these will be highly correlated with # of times a player gets to bat in their career.

(b) Using the all-possible-subsets approach, find the model that best fits the observed data. This procedure may be automated using the regsubsets() function in R, but you must explain in your own words how this algorithm identifies the ‘best' model. Note that you do not need to perform this task in Python.

Do not need to perform in Python.

(c) Using the forward-stepwise-selection approach, find the model that best fits the observed data. This procedure may be automated using the stepAIC() function in R, but you must explain in your own words how this algorithm identifies the ‘best’ model. Note that you do not need to perform this task in Python.

Do not need to perform in Python.

(d) Using the backward-stepwise-selection approach, find the model that best fits the observed data. This procedure may be automated using the stepAIC() function in R, but you must explain in your own words how this algorithm identifies the ‘best’ model. Note that you do not need to perform this task in Python.

Do not need to perform in Python.

(e) Using the hybrid-stepwise-selection approach, find the model that best fits the observed data. This procedure may be automated using the stepAIC() function in R, but you must explain in your own words how this algorithm identifies the ‘best’ model. Note that you do not need to perform this task in Python.

Do not need to perform in Python.

(f) In this part you will compare the predictive performance of four models:

* i. The full model with all 19 explanatory variables.
* ii. The optimal model identified in part (b).
* iii. The best model from parts (c)-(e) (i.e., the best stepwise-selection model).
* iv. The model that is considered optimal with respect to the Bayesian Information Criterion (BIC) which contains the variables AtBat, Hits, Walks, CRBI, Division and PutOuts.

Randomly split the observed data into a training set (containing roughly 80% of all of the data) and a held-out test set (containing roughly 20% of all of the data). Calculate the predictive root-mean-square error (RMSE) for each of the four models. Which model appears to be most appropriate? Justify why this model is most appropriate.

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
train = pd.concat([X_train, y_train], axis = 1)

#i. The full model with all 19 explanatory variables.
model1 = smf.ols('Salary ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + League + Division + NewLeague + PutOuts + Assists + Errors', data = train).fit()
pred1 = model1.predict(X_test)
RMSE1 = np.sqrt(np.mean((y_test - pred1)**2))
print('The predictive RMSE for the full model is', np.round(RMSE1, 4))

#ii. The optimal model identified in part (b).
model2 = smf.ols('Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + League + Division + PutOuts + Assists', data = train).fit()
pred2 = model2.predict(X_test)
RMSE2 = np.sqrt(np.mean((y_test - pred2)**2))
print('The predictive RMSE for the optimal model from part (b) is ', np.round(RMSE2, 4))

#iii. The best model from parts (c)-(e) (i.e., the best stepwise-selection model).
model3 = smf.ols('Salary ~ CRBI + Hits + PutOuts + Division + AtBat + Walks + CWalks + CRuns + CAtBat + Assists', data = train).fit()
pred3 = model3.predict(X_test)
RMSE3 = np.sqrt(np.mean((y_test - pred3)**2))
print('The predictive RMSE for the best model selected by stepwise selection is ', round(RMSE3, 4))

#iv. The model that is considered optimal with respect to the Bayesian Information Criterion (BIC) which contains the variables AtBat, Hits, Walks, CRBI, Division and PutOuts.
model4 = smf.ols('Salary ~ AtBat + Hits + Walks + CRBI + Division + PutOuts', data = train).fit()
pred4 = model4.predict(X_test)
RMSE4 = np.sqrt(np.mean((y_test - pred4)**2))
print('The predictive RMSE for the optimal model (in terms of BIC) is ', round(RMSE4, 4))

The predictive RMSE for the full model is 330.9044
The predictive RMSE for the optimal model from part (b) is  307.9706
The predictive RMSE for the best model selected by stepwise selection is  308.126
The predictive RMSE for the optimal model (in terms of BIC) is  286.7861


Based on the RMSE, the best predicitive model is the optimal BIC model since it has the smallest RMSE. In contrast, the worst predicitive model is the full model since it has the largest RMSE. And both the optimal model from part (b) and the stepwise selection mdoel perform similarly since the difference between both there RMSE values is very small and they perform only almost as well asn the optimal BIC model.

(g) As in part (f), you must compare the predictive performance of the same four models, but here you must determine the predictive accuracy (predictive RMSE) by using 10-Fold Cross Validation. Which model appears to be most appropriate? Justify why this model is most appropriate.

In [10]:
numfolds = 10
n = data.shape[0]

#i. The full model with all 19 explanatory variables.
kf = KFold(n=n, n_folds=numfolds, shuffle = True)
MSE = 0
for train_indices, test_indices in kf:
    train_X = X.iloc[train_indices, :]; train_y = y[train_indices]
    test_X = X.iloc[test_indices, :]; test_y = y[test_indices]
    training = pd.concat([train_X, train_y], axis = 1)
    m = smf.ols('Salary ~ AtBat + Hits + HmRun + Runs + RBI + Walks + Years + CAtBat + CHits + CHmRun + CRuns + CRBI + CWalks + League + Division + NewLeague + PutOuts + Assists + Errors', data = training).fit()   
    pred = m.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE1 = np.sqrt(MSE/numfolds)
print('The predictive RMSE for the full model is', np.round(RMSE1, 4))

#ii. The optimal model identified in part (b).
kf = KFold(n=n, n_folds=numfolds, shuffle = True)
MSE = 0
for train_indices, test_indices in kf:
    train_X = X.iloc[train_indices, :]; train_y = y[train_indices]
    test_X = X.iloc[test_indices, :]; test_y = y[test_indices]
    training = pd.concat([train_X, train_y], axis = 1)
    m = smf.ols('Salary ~ AtBat + Hits + Walks + CAtBat + CRuns + CRBI + CWalks + League + Division + PutOuts + Assists', data = training).fit()   
    pred = m.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE2 = np.sqrt(MSE/numfolds)
print('The predictive RMSE for the optimal model from part (b) is ', np.round(RMSE2, 4))

#iii. The best model from parts (c)-(e) (i.e., the best stepwise-selection model).
kf = KFold(n=n, n_folds=numfolds, shuffle = True)
MSE = 0
for train_indices, test_indices in kf:
    train_X = X.iloc[train_indices, :]; train_y = y[train_indices]
    test_X = X.iloc[test_indices, :]; test_y = y[test_indices]
    training = pd.concat([train_X, train_y], axis = 1)
    m = smf.ols('Salary ~ CRBI + Hits + PutOuts + Division + AtBat + Walks + CWalks + CRuns + CAtBat + Assists', data = training).fit()   
    pred = m.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE3 = np.sqrt(MSE/numfolds)
print('The predictive RMSE for the best model selected by stepwise selection is ', round(RMSE3, 4))

#iv. The model that is considered optimal with respect to the Bayesian Information Criterion (BIC) which contains the variables AtBat, Hits, Walks, CRBI, Division and PutOuts.
kf = KFold(n=n, n_folds=numfolds, shuffle = True)
MSE = 0
for train_indices, test_indices in kf:
    train_X = X.iloc[train_indices, :]; train_y = y[train_indices]
    test_X = X.iloc[test_indices, :]; test_y = y[test_indices]
    training = pd.concat([train_X, train_y], axis = 1)
    m = smf.ols('Salary ~ AtBat + Hits + Walks + CRBI + Division + PutOuts', data = training).fit()   
    pred = m.predict(test_X)
    MSE = MSE + np.mean((test_y - pred)**2)
RMSE4 = np.sqrt(MSE/numfolds)
print('The predictive RMSE for the optimal model (in terms of BIC) is ', round(RMSE4, 4))

The predictive RMSE for the full model is 344.2269
The predictive RMSE for the optimal model from part (b) is  329.0459
The predictive RMSE for the best model selected by stepwise selection is  330.5032
The predictive RMSE for the optimal model (in terms of BIC) is  328.7458


Based on the RMSE, the best predicitive model is the optimal BIC model since it has the smallest RMSE. In contrast, the worst predicitive model is the full model since it has the largest RMSE. And both the optimal model from part (b) and the stepwise selection mdoel perform similarly to the optimal BIC model.

(h) Given the estimates of predictive accuracy from parts (f) and (g) indicate which estimates you believe to be more accurate. In other words, indicate which validation approach (i.e., cross validation vs. k-fold cross validation) you believe will most accurately estimate the predictive capability of a model. Briefly explain your rationale.

In our case, we see that the 10-fold cross validation uses all of the data provided and giving us a more accurate predictive model. Therefore, the k-fold corss validation estimates are more accurate than cross validation since k-cross validation follows the 80-20 parition we want instead of individual partitions like in cross validation.

(i) Accounting for all of the analyses you’ve performed (i.e., multicollinearity, goodness-of-fit, and predictive accuracy), which model would you be most comfortable using? Briefly justify your choice. [Note: I’m not looking for a right or wrong answer here; I want to see that you can sensibly and eloquently justify your choice].

I'd be more comfortable using the optimal BIC model since we see in part (g) that it is the best predictive model when using 10-fold cross validation which we discussed in (h) has the best accuracy over cross-validation results. Another reason for this decision is that the optimal BIC model is also protected from multicollinearity compared to other models. This protection comes from the model only containing one of the multicollinear explanatory varibales.